In [9]:
###This notebook should be run only after a pubmed database has been created using the create_pubmed_db notebook

In [ ]:
import pickle
from transformers import *
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
def lst_top_keywords(lst_top_5_keywords,key,cos_sim):
    lst_top_5_keywords = sorted(lst_top_5_keywords, key=lambda x: x[1],reverse=False)
    if cos_sim > lst_top_5_keywords[0][1]:
        lst_top_5_keywords[0][0] = key
        lst_top_5_keywords[0][1] = cos_sim
    return lst_top_5_keywords

def add_to_lst_scientists(value_scientist,key,top_10_scientists):
    top_10_scientists = sorted(top_10_scientists, key=lambda x: x[1],reverse=False)
    if value_scientist > top_10_scientists[0][1]:
        top_10_scientists[0][0] = key
        top_10_scientists[0][1] = value_scientist
    return top_10_scientists

In [58]:
## dict_vectors is a dict of all keywords mentioned in the database and their associated Biobert embeddings
with open('keywords_and_vectors_v2.pkl', 'rb') as handle:
    dict_vectors = pickle.load(handle)
    
## dict_auhtors is a dict of all authors mentioned in the database with all the keywords associated with 
#the articles they've published and a count of mention for each keyword-author pair
with open('dict_authors.pickle', 'rb') as handle:
    dict_authors = pickle.load(handle)


In [6]:
scibert_tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
scibert_model = AutoModel.from_pretrained('dmis-lab/biobert-v1.1')

In [59]:
%%time
search_query = 'ivermectin covid'

token_keyword = torch.tensor(scibert_tokenizer.encode(search_query)).unsqueeze(0)
out = scibert_model(token_keyword)
vector = out[0][0][1:-1].detach().numpy().mean(axis=0)

lst_top_10_keywords = [['',0],['',0],['',0],['',0],['',0],['',0],['',0],['',0],['',0],['',0]]
for key,values in dict_vectors.items():
    if isinstance(vector,np.ndarray) and isinstance(values,np.ndarray):
        try:
            cos_sim = cosine_similarity(vector.reshape(1,-1),values.reshape(1,-1))[0][0]
            lst_top_5_keywords = lst_top_keywords(lst_top_10_keywords,key,cos_sim)
        except ValueError as e:
            print(e)
lst_top_10_keywords

Input contains NaN, infinity or a value too large for dtype('float32').
CPU times: user 17.9 s, sys: 67.6 ms, total: 18 s
Wall time: 17.8 s


[['covid lockdown', 0.9440769],
 ['covid lung', 0.9405372],
 ['covid management', 0.9384284],
 ['covid-19 fret substrate', 0.94154215],
 ['covid model', 0.9389711],
 ['boa boast covid-19 guidelines', 0.94477713],
 ['covid feet', 0.9467763],
 ['covid training', 0.93901384],
 ['covid emergency', 0.94047546],
 ['covid-19 hcws deaths', 0.94902503]]

In [60]:
lst_keywords = [keyword[0] for keyword in lst_top_10_keywords if keyword[1]>0.9]

top_10_scientists = [['',0],['',0],['',0],['',0],['',0],['',0],['',0],['',0],['',0],['',0]]
for key,values in dict_authors.items():
    value_scientist = 0
    for keyword in lst_keywords:
        if keyword in values:
            value_scientist += values[keyword]
    top_10_scientists = add_to_lst_scientists(value_scientist,key,top_10_scientists)
    

In [61]:
for scientist in top_10_scientists:
    print(scientist[0], dict_authors[scientist[0]])

Luca Damiani {'ar surgery': 1, 'covid emergency': 1, 'augmented reality': 1, 'hologram 3d display': 1, 'remote assistance': 1, 'remote proctor': 1, 'spine surgery': 1, 'telementoring and surgery': 1}
Matteo Cavalieri {'ar surgery': 1, 'covid emergency': 1, 'augmented reality': 1, 'hologram 3d display': 1, 'remote assistance': 1, 'remote proctor': 1, 'spine surgery': 1, 'telementoring and surgery': 1}
Nicola Zullo {'ar surgery': 1, 'covid emergency': 1, 'augmented reality': 1, 'hologram 3d display': 1, 'remote assistance': 1, 'remote proctor': 1, 'spine surgery': 1, 'telementoring and surgery': 1}
Francesco Zenga {'ar surgery': 1, 'covid emergency': 1, 'augmented reality': 1, 'hologram 3d display': 1, 'remote assistance': 1, 'remote proctor': 1, 'spine surgery': 1, 'telementoring and surgery': 1}
Nicola Marengo {'ar surgery': 1, 'covid emergency': 1, 'augmented reality': 1, 'hologram 3d display': 1, 'remote assistance': 1, 'remote proctor': 1, 'spine surgery': 1, 'telementoring and surg